### Scraping Data Tokopedia

In [2]:
library("rvest")
library("dplyr")
library(stringr)
library(readr)

In [3]:
scrape<-function(keywords,page)
{
  key = gsub(" ","%20",keywords)
  all_data<-data.frame()
  for (x in 1:page) 
  {
      url<-paste("https://www.tokopedia.com/search?navsource=home&page=",x,"&q=",key,sep="")
      readUrl<-read_html(url)

      product<- readUrl %>%
           html_nodes('div[data-testid=spnSRPProdName]') %>%
           html_text()

      price<- readUrl %>%
           html_nodes('div[data-testid=spnSRPProdPrice]')%>%
           html_text()

      city<- readUrl %>%
           html_nodes('span[data-testid=spnSRPProdTabShopLoc]')%>%
           html_text()     

      store<-c() 
      att<-readUrl %>%
           html_nodes('span[class="css-1kdc32b flip"]') %>%
           html_text() %>%
           str_replace_all("&#39;","'")
      count = 1
      for(i in att)
          {
              if(count %% 2 == 0)
                  {
                      store<-c(store,i)
                  }
              count = count + 1
          }

       link<-readUrl%>%
        html_nodes('div[class="css-12sieg3"]') %>%
        html_nodes('div[class="css-y5gcsw"]') %>%
        html_nodes('div[class="css-5fmc3z"]') %>%
        html_nodes('div[class="css-qa82pd"]') %>%
        html_nodes('div[class="css-1c4umxf"]') %>%
        html_nodes('div[class="pcv3__container css-gfx8z3"]') %>%
        html_nodes('div[class="css-zimbi"]') %>%
        html_nodes('a[href]') %>%
        html_attr('href') %>%
        str_remove(".*www.tokopedia.com") %>%
        str_replace_all(c("%2F"="/","%3D"="=")) %>%
        paste0("https://www.tokopedia.com",.,sep="") %>%
        str_remove("%3F.*") %>%
        str_remove("\\?.*")
          info<-data.frame()
          count <- 1
          for(index in link)
            {
              kondisi<-read_html(index) %>%
              html_nodes('ul[data-testid="lblPDPInfoProduk"] [class="css-1dmo88g"]') %>%
              html_text() %>% strsplit(split = ": ")
              kondisi<-kondisi[grep('Kondisi',kondisi)][[1]][2] 
              
              berat<-read_html(index) %>%
              html_nodes('ul[data-testid="lblPDPInfoProduk"] [class="css-1dmo88g"]') %>%
              html_text() %>% strsplit(split = ": ")
              berat<-berat[grep('Berat',berat)][[1]][2]
              
              kategori<-read_html(index)%>%
              html_nodes('ul[data-testid="lblPDPInfoProduk"] [class="css-1dmo88g"]') %>%
              html_text() %>% strsplit(split = ": ")
              kategori<-kategori[grep('Kategori',kategori)][[1]][2]
              
              jml_ulasan<-read_html(index)%>%
              html_nodes('div,span[data-testid="lblPDPDetailProductRatingCounter"]') %>%
              html_nodes('.css-ddjk3l') %>%
              html_text() %>% str_remove('.css-1s46cvt.*') %>%
              parse_number()
              
              count = count + 1
              info<-rbind(info,data.frame(kondisi,berat,kategori,jml_ulasan))
            }
      
      promo<-readUrl%>%
        html_nodes('div[class="css-12sieg3"]') %>%
        html_nodes('div[class="css-y5gcsw"]') %>%
        html_nodes('div[class="css-5fmc3z"]') %>%
        html_nodes('div[class="css-qa82pd"]') %>%
        html_nodes('div[class="css-1c4umxf"]') %>%
        html_nodes('div[class="pcv3__container css-gfx8z3"]') %>%
        html_nodes('div[class="css-974ipl"]') %>%
        html_nodes('a[class="pcv3__info-content css-gwkf0u"]') %>%
        lapply(. %>% html_nodes('[class="css-tolj34"]') %>% html_text() %>%
        ifelse(identical(., character(0)), NA, .)) %>% unlist

      rating<-readUrl%>%
        html_nodes('div[class="css-12sieg3"]') %>%
        html_nodes('div[class="css-y5gcsw"]') %>%
        html_nodes('div[class="css-5fmc3z"]') %>%
        html_nodes('div[class="css-qa82pd"]') %>%
        html_nodes('div[class="css-1c4umxf"]') %>%
        html_nodes('div[class="pcv3__container css-gfx8z3"]') %>%
        html_nodes('div[class="css-974ipl"]') %>%
        html_nodes('a[class="pcv3__info-content css-gwkf0u"]') %>%
        lapply(. %>% html_nodes('[class="css-t70v7i"]') %>% html_text() %>%
           ifelse(identical(., character(0)), NA, .)) %>% unlist

      sold<-readUrl%>%
        html_nodes('div[class="css-12sieg3"]') %>%
        html_nodes('div[class="css-y5gcsw"]') %>%
        html_nodes('div[class="css-5fmc3z"]') %>%
        html_nodes('div[class="css-qa82pd"]') %>%
        html_nodes('div[class="css-1c4umxf"]') %>%
        html_nodes('div[class="pcv3__container css-gfx8z3"]') %>%
        html_nodes('div[class="css-974ipl"]') %>%
        html_nodes('a[class="pcv3__info-content css-gwkf0u"]') %>%
        lapply(. %>% html_nodes('[class="css-1duhs3e"]') %>% html_text() %>%
           ifelse(identical(., character(0)), NA, .)) %>% unlist

        all_data<-rbind(all_data,data.frame(product,city,store,price,promo,rating,sold,link,info))
  } 
    
  all_data<- all_data %>%
           mutate(no=1:nrow(all_data), .before=product) %>%
           mutate(price = str_replace(price, "Rp", "")) %>%
           mutate(price = as.numeric(gsub(",", ".", gsub("\\.", "", price)))) %>%
           mutate(product = str_to_title(product)) %>%
           mutate(product = gsub("  "," ",product))
    
  return(all_data)
}

In [4]:
#scrape('input keywords',input total page number)
output<-scrape('Samsung M31',5)

In [5]:
output

no,product,city,store,price,promo,rating,sold,link,kondisi,berat,kategori,jml_ulasan
<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Samsung Galaxy A03 Ram 4/64 Gb - Kamera 48 Mp - Blue 3/32gb,Jakarta Utara,TMB Ponsel,1430000,Cashback,4.9,Terjual 1 rb+,https://www.tokopedia.com/tmb-ponsel/samsung-galaxy-a03-ram-4-64-gb-kamera-48-mp-blue-3-32gb-ac76,Baru,500 Gram,Android OS,418
2,Samsung M31 Soft Case Silikon Scrub Sandstone Keset Slim,Jakarta Barat,NICE CASE,33000,NA,5.0,Terjual 100+,https://www.tokopedia.com/nicecase/samsung-m31-soft-case-silikon-scrub-sandstone-keset-slim,Baru,70 Gram,Soft Case Handphone,59
3,Samsung Galaxy A53 5g 8/256gb - Phantom White,Surabaya,Las Vegas Mobile Store - Samsung Authorized,6499000,NA,NA,NA,https://www.tokopedia.com/lasvegas-samsung/samsung-galaxy-a53-5g-8-256gb-phantom-white,Baru,1.000 Gram,Android OS,0
4,Case Samsung Galaxy M31 Evoskin Anti Skid Silicone,Bekasi,Caselova Store,29000,Cashback,4.9,Terjual 80+,https://www.tokopedia.com/caselovastore/case-samsung-galaxy-m31-evoskin-anti-skid-silicone-hitam,Baru,50 Gram,Soft Case Handphone,53
5,Samsung Galaxy A53 5g 8/256gb - Phantom Blue,Surabaya,Las Vegas Mobile Store - Samsung Authorized,6499000,NA,NA,Terjual 1,https://www.tokopedia.com/lasvegas-samsung/samsung-galaxy-a53-5g-8-256gb-phantom-blue,Baru,1.000 Gram,Android OS,0
6,Samsung M31 6/128,Depok,marsjaya,3299990,NA,4.9,Terjual 100+,https://www.tokopedia.com/marsaliando/samsung-m31-6-128-m32-putih-8-128-6b7b,Baru,500 Gram,Android OS,55
7,Samsung M31 Ex Sein Termurah,Jakarta Barat,JJStore.ID,2599000,Cashback,5.0,Terjual 30+,https://www.tokopedia.com/jjstoreid/samsung-m31-ex-sein-termurah,Bekas,500 Gram,Android OS,9
8,Samsung M31 6/128gb Fullset Ori Mulus Net,Jakarta Pusat,pho pho cell,2300000,NA,4.9,Terjual 30+,https://www.tokopedia.com/didoalfarezi/samsung-m31-6-128gb-fullset-ori-mulus-net,Bekas,450 Gram,Android OS,12
9,Samsung M31 6/128 Gb Garansi Sein Indonesia Second Bekas Seken Ori,Jakarta Utara,VstoreINDO,2349000,NA,5.0,Terjual 15,https://www.tokopedia.com/vstoreindo/samsung-m31-6-128-gb-garansi-sein-indonesia-second-bekas-seken-ori-unit-only,Bekas,300 Gram,Android OS,8
